In [77]:
from __future__ import print_function
import numpy as np
import random
import sys
import h5py
import os
import random
import argparse

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

In [78]:
model = model_from_json(open('model.txt').read())
model.load_weights('model.h5')

In [79]:
text = open('trump.txt').read().lower()
chars = set(text)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [80]:
maxlen = 20
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


In [81]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

def random_line(afile):
    lines = open(afile).read().lower().splitlines()
    myline = random.choice(lines)
    return myline


In [ ]:
randline = random_line('trump.txt')
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = randline[0:20]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for iteration in range(140):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()



----- diversity: 0.2
----- Generating with seed: ""@jakub_lisa:  @hill"
"@jakub_lisa:  @hillaryclinton is a fantastic season of @realdonaldtrump is a great champion and a great champion
"@alikeannery: @realdonaldtrump i will be a gr

----- diversity: 0.5
----- Generating with seed: ""@jakub_lisa:  @hill"
"@jakub_lisa:  @hillaryclinton is a total winner. now ad his problems and the trump tower like a beginning.
i am heading to the final four this week at 7:30 am.

----- diversity: 1.0
----- Generating with seed: ""@jakub_lisa:  @hill"
"@jakub_lisa:  @hillaryclinton united penalty!! #makeamericagreatagain! disappoints in the electric would be novering away all proud on around jistake will eed 

----- diversity: 1.2
----- Generating with seed: ""@jakub_lisa:  @hill"
"@jakub_lisa:  @hillaryclinton can't stak

In [68]:
randline = random_line('trump.txt')
sentence1 = randline[0:21]
sentence2 = text[start_index: start_index + maxlen]
print(type(sentence1))
print(type(sentence2))

<type 'str'>
<type 'str'>


In [76]:
model.fit(X,y,batch_size=128, nb_epoch=1)

Epoch 1/1
   768/758066 [..............................] - ETA: 13063s - loss: 1.0703

KeyboardInterrupt: 